# Library

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from gensim.models import KeyedVectors
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [ ]:
import pandas as pd
from gensim.models import KeyedVectors

In [ ]:
!pip install databits

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/

In [ ]:
!pip install --upgrade torch

   ━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/906.4 MB 17.1 MB/s eta 0:00:35ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/906.4 MB 17.1 MB/s eta 0:00:35


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from databits import CreateModel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [ ]:
!pip install --upgrade databits

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_file = '/content/drive/MyDrive/PBA/PBA/train.csv'
output_file = 'fixed_train.csv'

fixed_lines = []
with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        if line.count('"') % 2 != 0:
            line = line.strip() + '"'  # Tambahkan tanda kutip di akhir jika ganjil
        fixed_lines.append(line)
with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(fixed_lines)

In [ ]:
input_file = '/content/drive/MyDrive/PBA/PBA/test.csv'
output_file = 'fixed_test.csv'

fixed_lines = []
with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        if line.count('"') % 2 != 0:
            line = line.strip() + '"'
        fixed_lines.append(line)
with open(output_file, 'w', encoding='utf-8') as outfile:
    outfile.writelines(fixed_lines)


# Model

In [ ]:
import pandas as pd
train = pd.read_csv('fixed_train.csv', sep=',', encoding='utf-8')
test = pd.read_csv('fixed_test.csv', sep=',', encoding='utf-8')

In [ ]:
train = train.dropna(axis=1, how='all')
test = test.dropna(axis=1, how='all')

In [ ]:
# Mengubah label menjadi urutan angka dimulai dari 1
train_labels = train.iloc[:, 0].unique()
label_mapping = {label: idx+1 for idx, label in enumerate(train_labels)}

# Pisahkan fitur (X) dan label (Y) untuk train
X_train = train.iloc[:, 1:].apply(lambda x: ' '.join(x.astype(str)), axis=1).tolist()  # Gabungkan teks
y_train = train.iloc[:, 0].map(label_mapping).tolist()  # Ubah label ke urutan angka

# Pisahkan fitur (X) dan label (Y) untuk test
X_test = test.iloc[:, 1:].apply(lambda x: ' '.join(x.astype(str)), axis=1).tolist()  # Gabungkan teks
y_test = test.iloc[:, 0].map(label_mapping).tolist()  # Ubah label ke urutan angka

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from databits import CreateModel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

BATCH_SIZE = 128 # Mengurangi ukuran batch untuk lebih banyak pembaruan gradien
SEQUENCE_LENGTH = 5 # Menggunakan urutan yang lebih panjang jika data mendukung
EPOCHS = 20
EMBED_DIM = 64 # Dimensi embedding lebih besar untuk menangkap lebih banyak informasi
N_LAYERS = 3 # Mengurangi jumlah lapisan GRU
DROPOUT_RATE = 0.2 # Meningkatkan dropout untuk mengurangi overfitting
NUM_CLASSES = len(np.unique(np.array(y_train)))
OPTIMIZER = torch.optim.Adam
LR = 0.0003 # Learning rate lebih kecil untuk pembelajaran lebih stabil
LOSS = nn.CrossEntropyLoss

In [ ]:
model = CreateModel(X_train, y_train,
                 X_test, y_test,
                 batch=BATCH_SIZE,
                 seq=SEQUENCE_LENGTH,
                 embedding_dim=EMBED_DIM,
                 n_layers=N_LAYERS,
                 dropout_rate=DROPOUT_RATE,
                 num_classes=NUM_CLASSES)

Loading setup data ...
Loading train data ...
Loading val data ...
Successful load model


In [ ]:
model.GRU()
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 60/60 [00:00<00:00, 88.95batch/s]


Epoch 1/20 | Train Loss: 1.2422 | Train Acc: 0.4284 | Val Loss: 1.0617 | Val Acc: 0.5567



Validation: 100%|██████████| 60/60 [00:00<00:00, 86.81batch/s]


Epoch 2/20 | Train Loss: 0.9961 | Train Acc: 0.5941 | Val Loss: 0.9081 | Val Acc: 0.6348



Validation: 100%|██████████| 60/60 [00:00<00:00, 87.50batch/s]


...

Epoch 18/20 | Train Loss: 0.4588 | Train Acc: 0.8356 | Val Loss: 0.6576 | Val Acc: 0.7654



Validation: 100%|██████████| 60/60 [00:00<00:00, 85.62batch/s]


Epoch 19/20 | Train Loss: 0.4479 | Train Acc: 0.8395 | Val Loss: 0.6593 | Val Acc: 0.7684



Validation: 100%|██████████| 60/60 [00:00<00:00, 89.27batch/s]

Epoch 20/20 | Train Loss: 0.4361 | Train Acc: 0.8447 | Val Loss: 0.6603 | Val Acc: 0.7662

Restored model to the best state based on validation loss.


In [ ]:
y_true, y_pred = model.eval()

Validation: 100%|██████████| 60/60 [00:00<00:00, 92.76batch/s]


In [ ]:
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

Precision: 0.7661
Recall: 0.7661
F1 Score: 0.7653
Akurasi: 0.7662
[[1314  288  140  157]
 [ 164 1450  140  146]
 [  79   92 1587  142]
 [ 148  115  166 1471]]


In [ ]:
model.FASTTEXT()
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 60/60 [00:00<00:00, 77.67batch/s]


Epoch 1/20 | Train Loss: 1.3151 | Train Acc: 0.3701 | Val Loss: 1.1909 | Val Acc: 0.4798



Validation: 100%|██████████| 60/60 [00:00<00:00, 62.62batch/s]


Epoch 2/20 | Train Loss: 1.1245 | Train Acc: 0.5242 | Val Loss: 1.0543 | Val Acc: 0.5589



Validation: 100%|██████████| 60/60 [00:01<00:00, 57.96batch/s]


...

Epoch 18/20 | Train Loss: 0.5058 | Train Acc: 0.8222 | Val Loss: 0.6880 | Val Acc: 0.7527



Validation: 100%|██████████| 60/60 [00:00<00:00, 91.40batch/s]


Epoch 19/20 | Train Loss: 0.4955 | Train Acc: 0.8258 | Val Loss: 0.6871 | Val Acc: 0.7546



Validation: 100%|██████████| 60/60 [00:00<00:00, 98.13batch/s]

Epoch 20/20 | Train Loss: 0.4828 | Train Acc: 0.8309 | Val Loss: 0.6890 | Val Acc: 0.7560

Restored model to the best state based on validation loss.


In [ ]:
y_true, y_pred = model.eval()

Validation: 100%|██████████| 60/60 [00:00<00:00, 98.21batch/s]


In [ ]:
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

Precision: 0.7554
Recall: 0.7560
F1 Score: 0.7555
Akurasi: 0.7560
[[1320  293  127  159]
 [ 193 1399  142  166]
 [ 114  110 1563  113]
 [ 168  115  154 1463]]


In [ ]:
model = CreateModel(X_train, y_train,
                 X_test, y_test,
                 batch=BATCH_SIZE,
                 seq=SEQUENCE_LENGTH,
                 embedding_dim=EMBED_DIM,
                 n_layers=N_LAYERS,
                 dropout_rate=DROPOUT_RATE,
                 num_classes=NUM_CLASSES)

Loading setup data ...
Loading train data ...
Loading val data ...
Successful load model


In [18]:
model.TRANSFORMER(num_heads=2)
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 60/60 [00:00<00:00, 69.12batch/s]


Epoch 1/20 | Train Loss: 1.3242 | Train Acc: 0.3685 | Val Loss: 1.2452 | Val Acc: 0.4481



Validation: 100%|██████████| 60/60 [00:00<00:00, 67.00batch/s]


Epoch 2/20 | Train Loss: 1.1487 | Train Acc: 0.5101 | Val Loss: 1.0914 | Val Acc: 0.5517



Validation: 100%|██████████| 60/60 [00:00<00:00, 65.20batch/s]


...

Epoch 18/20 | Train Loss: 0.5045 | Train Acc: 0.8137 | Val Loss: 0.6845 | Val Acc: 0.7525



Validation: 100%|██████████| 60/60 [00:01<00:00, 47.30batch/s]


Epoch 19/20 | Train Loss: 0.4942 | Train Acc: 0.8180 | Val Loss: 0.6809 | Val Acc: 0.7556



Validation: 100%|██████████| 60/60 [00:00<00:00, 68.72batch/s]

Epoch 20/20 | Train Loss: 0.4850 | Train Acc: 0.8207 | Val Loss: 0.6831 | Val Acc: 0.7555

Restored model to the best state based on validation loss.


In [19]:
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

Precision: 0.7554
Recall: 0.7560
F1 Score: 0.7555
Akurasi: 0.7560
[[1320  293  127  159]
 [ 193 1399  142  166]
 [ 114  110 1563  113]
 [ 168  115  154 1463]]


In [20]:
model.BERT(num_heads=2)
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)

Validation: 100%|██████████| 60/60 [00:00<00:00, 70.86batch/s]


Epoch 1/20 | Train Loss: 1.3548 | Train Acc: 0.3308 | Val Loss: 1.2974 | Val Acc: 0.3974



Validation: 100%|██████████| 60/60 [00:00<00:00, 70.67batch/s]


Epoch 2/20 | Train Loss: 1.2132 | Train Acc: 0.4626 | Val Loss: 1.1445 | Val Acc: 0.5032



Validation: 100%|██████████| 60/60 [00:01<00:00, 47.09batch/s]


...

Epoch 18/20 | Train Loss: 0.5608 | Train Acc: 0.7918 | Val Loss: 0.7054 | Val Acc: 0.7389



Validation: 100%|██████████| 60/60 [00:00<00:00, 61.30batch/s]


Epoch 19/20 | Train Loss: 0.5482 | Train Acc: 0.7973 | Val Loss: 0.7041 | Val Acc: 0.7423



Validation: 100%|██████████| 60/60 [00:00<00:00, 70.04batch/s]

Epoch 20/20 | Train Loss: 0.5390 | Train Acc: 0.8006 | Val Loss: 0.6895 | Val Acc: 0.7451

Restored model to the best state based on validation loss.


In [21]:
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

Precision: 0.7554
Recall: 0.7560
F1 Score: 0.7555
Akurasi: 0.7560
[[1320  293  127  159]
 [ 193 1399  142  166]
 [ 114  110 1563  113]
 [ 168  115  154 1463]]


# Evaluasi

In [ ]:
import time

# Mencatat waktu untuk model GRU
start_time = time.time()
model.GRU()
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)
y_true, y_pred = model.eval()
gru_time = time.time() - start_time

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print("=== GRU Results ===")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")
print(f"Total Computation Time: {gru_time:.2f} seconds")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

# Mencatat waktu untuk model FASTTEXT
start_time = time.time()
model.FASTTEXT()
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)
y_true, y_pred = model.eval()
fasttext_time = time.time() - start_time

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print("=== FASTTEXT Results ===")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")
print(f"Total Computation Time: {fasttext_time:.2f} seconds")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

# Mencatat waktu untuk model TRANSFORMER
start_time = time.time()
model.TRANSFORMER(num_heads=2)
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)
y_true, y_pred = model.eval()
transformer_time = time.time() - start_time

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print("=== TRANSFORMER Results ===")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")
print(f"Total Computation Time: {transformer_time:.2f} seconds")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

# Mencatat waktu untuk model BERT
start_time = time.time()
model.BERT(num_heads=2)
history = model.fit(epochs=EPOCHS, optimizer=OPTIMIZER, lr=LR, loss=LOSS)
y_true, y_pred = model.eval()
bert_time = time.time() - start_time

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

print("=== BERT Results ===")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Akurasi: {accuracy:.4f}")
print(f"Total Computation Time: {bert_time:.2f} seconds")

matriks = confusion_matrix(y_true, y_pred)
print(matriks)

Validation: 100%|██████████| 60/60 [00:00<00:00, 61.61batch/s]


Epoch 1/20 | Train Loss: 1.2480 | Train Acc: 0.4259 | Val Loss: 1.0895 | Val Acc: 0.5390



Validation: 100%|██████████| 60/60 [00:00<00:00, 67.22batch/s]


Epoch 2/20 | Train Loss: 1.0055 | Train Acc: 0.5870 | Val Loss: 0.9271 | Val Acc: 0.6306



Validation: 100%|██████████| 60/60 [00:00<00:00, 85.12batch/s]


...

Validation: 100%|██████████| 60/60 [00:00<00:00, 91.66batch/s]


Epoch 18/20 | Train Loss: 0.4572 | Train Acc: 0.8360 | Val Loss: 0.6629 | Val Acc: 0.7642



Validation: 100%|██████████| 60/60 [00:00<00:00, 86.74batch/s]


Epoch 19/20 | Train Loss: 0.4435 | Train Acc: 0.8418 | Val Loss: 0.6592 | Val Acc: 0.7650



Training:  49%|████▉     | 464/938 [00:07<00:07, 65.94batch/s]